# Pickup Line Generator

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Creating a ML folder for all machine learning purposes

and a pickup folder for this project


In [ ]:
%cd /content/drive/My\ Drive
%mkdir ML
%cd ML
!ls

%mkdir pickup
!ls

/content/drive/My Drive
mkdir: cannot create directory ‘ML’: File exists
/content/drive/My Drive/ML
pickup
mkdir: cannot create directory ‘pickup’: File exists
pickup


In [ ]:
%cd /content/drive/My\ Drive/ML/pickup

/content/drive/My Drive/ML/pickup


Setting up to save model to cloud

In [ ]:
!sudo apt-get install git-lfs
!git clone https://huggingface.co/HJK/PickupLineGenerator 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,845 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 146374 files and directories c

Global imports

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 16.2MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 2.9MB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3e32537afae24bc6645c9e8bd38098412c80182950fe0c5f41c754522ed44c25
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import json
import urllib3
import pathlib
import shutil
import requests
import os
import re
import random

Variables

In [ ]:
# Have global access
trainer = None
tokenizer = None

# HYPER-PARAMETERS
ALLOW_NEW_LINES = False     
LEARNING_RATE = 1.372e-4
EPOCHS = 4

Create and save pickup line dataset

In [ ]:
def createData():
  f = 'data.txt'
  data = []

  with open(f,'r') as dtf:
      data = dtf.readlines()

  return cleanData(data)

def cleanData(data):
  for i in range(len(data)):
    data[i] = fix_text(data[i].strip())

  return data

def fix_text(text):
  text = text.replace('&amp;', '&')
  text = text.replace('&lt;', '<')
  text = text.replace('&gt;', '>')
  return text


def saveData():
  try:
    # create dataset
    data = createData()

    print(data[:5])

    # create a file based on multiple epochs with tweets mixed up
    seed_data = random.randint(0,2**32-1)
    dataRandom = random.Random(seed_data)
    total_text = '<|endoftext|>'
    for _ in range(EPOCHS):
        dataRandom.shuffle(data)
        total_text += '<|endoftext|>'.join(data) + '<|endoftext|>'
    
    if len(total_text) / EPOCHS < 6000:
        # need about 4000 chars for one data sample (but depends on spaces, etc)
        raise ValueError(f"Error: This doesn't seem like enough text")

    with open(f'datatrain.txt', 'w') as f:
        f.write(total_text)

  except Exception as e:
    print('\nAn error occured in saving the dataset...\n')
    print(e)

Finetuning on pickup line data


In [ ]:
def finetune():
  import transformers
  from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TextDataset, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments,
    get_cosine_schedule_with_warmup)

  try:                
    # Setting up pre-trained neural network
    global trainer, tokenizer
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    model = AutoModelForCausalLM.from_pretrained('gpt2', cache_dir=pathlib.Path('cache').resolve())
    block_size = tokenizer.model_max_length
    train_dataset = TextDataset(tokenizer=tokenizer, file_path=f'datatrain.txt', block_size=block_size, overwrite_cache=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    seed = random.randint(0,2**32-1)
    training_args = TrainingArguments(
      output_dir=f'output/',
      overwrite_output_dir=True,
      do_train=True,
      num_train_epochs=1,
      per_device_train_batch_size=1,
      prediction_loss_only=True,
      logging_steps=5,
      save_steps=0,
      seed=seed,
      learning_rate = LEARNING_RATE)

    # Set up Trainer
    trainer = Trainer(
      model=model,
      tokenizer=tokenizer,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset)
    
    # Update lr scheduler
    train_dataloader = trainer.get_train_dataloader()
    num_train_steps = len(train_dataloader)
    trainer.create_optimizer_and_scheduler(num_train_steps)
    trainer.lr_scheduler = get_cosine_schedule_with_warmup(
      trainer.optimizer,
      num_warmup_steps=0,
      num_training_steps=num_train_steps)
    
    # Training neural network
    trainer.train()

    # set model config parameters
    trainer.model.config.task_specific_params['text-generation'] = {
      'do_sample': True,
      'min_length': 10,
      'max_length': 160,
      'temperature': 1.,
      'top_p': 0.95,
      'prefix': '<|endoftext|>'}

    # save new model files
    trainer.save_model("pickup")

  except Exception as e:
    print('\nAn error occured in fine tuning...\n')
    print(e)

Predicting


In [ ]:
def shorten_text(text, max_char):
  while len(text) > max_char:
      text = ' '.join(text.split()[:-1]) + '…'
  return text
  
def predict(start):
  # get start sentence
  start = start.strip()

  predictions = []
  try:
    # prepare input
    start_with_bos = '<|endoftext|>' + start
    encoded_prompt = trainer.tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
    encoded_prompt = encoded_prompt.to(trainer.model.device)

    # prediction
    output_sequences = trainer.model.generate(
      input_ids=encoded_prompt,
      max_length=160,
      min_length=10,
      temperature=1.,
      top_p=0.95,
      do_sample=True,
      num_return_sequences=10
      )
    generated_sequences = []

    # decode prediction
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      generated_sequence = generated_sequence.tolist()
      text = trainer.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
      if not ALLOW_NEW_LINES:
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
      generated_sequences.append(text.strip())
    
    for i, g in enumerate(generated_sequences):
      predictions.append([start, g])
    
    for i, g in enumerate(generated_sequences):
      pred = start + ' → ' + g[len(start):].strip()
      print(pred)

  except Exception as e:
    print('\nAn error occured in predicting...\n')
    print(e)

Train

In [ ]:
saveData()
finetune()

['Nice pants. Can I test the zipper?', "You're like my menorah's candles…getting hotter every day.", 'I still exist, in case you have forgotten.', "If a fat man puts you in a bag at night, don't worry I told Santa I wanted you for Christmas.", "You know, if I were you, I'd have sex with me."]


/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (174237 > 1024). Running this sequence through the model will result in indexing errors


Step,Training Loss
5,3.832800
10,3.375600
15,3.302700
20,3.264000
25,3.192100
30,3.329700
35,2.933000
40,3.075000
45,2.898600
50,2.900200


Upload to cloud


In [ ]:
%cd /content/drive/My\ Drive/ML/pickup/PickupLineGenerator
trainer.save_model("./")
tokenizer.save_pretrained("./")

'''
!git config --global user.email "hjk.businessemail@gmail.com"
!git config --global user.name "HJK"
!git add --all
!git status
!git commit -m "version 0.1"
!git push
'''
# couldn't get this to work, just download and use git on your computer to upload

/content/drive/My Drive/ML/pickup/PickupLineGenerator
On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://huggingface.co': No such device or address


Predicting Demo



In [ ]:
predict("I love")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I love → you like a nerd loves socks.
I love → you. You've been my butt ever since, I know you've been a joy to find my heart. Want to fuck?
I love → you as an artist, yet don't know you want to know how beautiful he is?
I love → you like I could ever imagine!
I love → you like Mickey Mouse loves candy.
I love → you like a squirrel loves a duck.
I love → you like a baby loves his tongue.
I love → you because you remind me of a piece of art that was made in my honor.
I love → you. We have a few more hours until my name comes up.
I love → you like my sister loved the Titanic.
